# Preprocess NCBI Yellow Fever (YFV) Sequences

In this notebook,we will:
- Standardize the reference sequence FASTA definition lines
- Remove the gaps inserted in aligned reference sequences (`-` characters)
- Generate a set of simulated reads by using ARt Illumina.

> Note: The notebook was built and tested to run locally. It will not work on Colab or Kaggle because Art Illumina must be installed on the system.

# 1. Imports and setup environment

### Install and import packages

In [1]:
# Install required custom packages if not installed yet.
import importlib.util
if not importlib.util.find_spec('eccore'):
    print('installing package: `eccore`')
    ! pip install -qqU eccore
else:
    print('`eccore` already installed')
if not importlib.util.find_spec('metagentorch'):
    print('installing package: `metagentorch')
    ! pip install -qqU metagentorch
else:
    print('`metagentorch` already installed')

`eccore` already installed
`metagentorch` already installed


In [2]:
# Import all required packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re

from eccore.core import files_in_tree
from eccore.ipython import nb_setup
from IPython.display import display, Markdown
from nbdev import show_doc
from pathlib import Path
from pprint import pprint
from tqdm.notebook import tqdm, trange

# Setup the notebook for development
nb_setup()

from metagentorch.cnn_virus.data import FastaFileReader, FastqFileReader, AlnFileReader
from metagentorch.core import ProjectFileSystem
from metagentorch.art import ArtIllumina

Set autoreload mode


# 2. Review project file system

This project adopts a unified file structure to make coding and colaboration easier. In addition, we can run the code locally (from a `project-root` directory) or in the cloud (colab or kaggle).

The unified file structure when running localy is:
```text
    project-root   
        |--- data
        |      |--- CNN_Virus_data  (all data related to CNN Virus original paper)
        |      |--- ncbi            (all data related to NCBI reference sequences)
        |      |--- saved           (trained and finetuned models)
        |      |--- ....            (raw and pre-processed data from various sources)  
        |      
        |--- nbs  (all reference and work notebooks)
        |      |--- cnn_virus
        |      |        |--- ref_n.n_notebooks.ipynb
        |      |        |--- wip_n.n_notebooks.ipynb
```

When running on google colab, it is assumed that a google drive is mounted on the colab server instance, and that the google drive root includes a shortcut named `Metagenomics` and pointing to the project shared directory. The project shared directory is accessible [here](/https://drive.google.com/drive/folders/134uei5fmt08TpzhmjG4sW0FQ06kn2ZfZ) if your google account was given access permission.

To make access to the unified file system, `metagentorch` provides a `ProjectFileSystem` class (see [documentation](https://vtecftwy.github.io/metagentorch/core.html#projectfilesystem) for more info).

Key folders and system information

In [3]:
pfs = ProjectFileSystem()
pfs.info()

Running linux on local computer
Device's home directory: /home/vtec
Project file structure:
 - Root ........ /home/vtec/projects/bio/metagentorch 
 - Data Dir .... /home/vtec/projects/bio/metagentorch/data 
 - Notebooks ... /home/vtec/projects/bio/metagentorch/nbs


In [4]:
pfs.readme()

ReadMe file for directory `data`:

### Data structure for this project
This directory includes all the data required for the project.

```text
data
 |--- CNN_Virus_data 
 |--- ncbi                
 |--- saved         
 |--- yf-reads
 |--- ....           
     
```
#### Sub-directories
- `CNN_Virus_data`: includes all the data related to the original CNN Virus paper, i.e. training data and validation data in a format that can be used by the CNN Virus code.
- `ncbi`: includes data related to the use of viral sequences from NCBI: reference sequences, simulated reads, inference datasets, inference results.
- `saved`: includes model saved parameters and preprocessing datasets.
- `yf-reads`: includes all data related to real yellow fever reads, from "wet" samples

Also available on AWS S3 at `https://s3.ap-southeast-1.amazonaws.com/bio.cnn-virus.data/data/...`

In [5]:
files_in_tree(pfs.data/'ncbi/refsequences/yf');

refsequences
  |--yf
  |    |--yf_1971_angola.fa (0)
  |    |--yf_2023_yellow_fever.fa (1)
  |    |--yf_2023_yellow_fever_with_missing_bases.fa (2)
  |    |--yf_2023_yellow_fever_aligned.fa (3)
  |    |--yf_2023_multiple_alignment_original.fa (4)
  |    |--readme.md (5)
  |    |--yf_NC_002031_full_sequence.fa (6)


`yf_2023_multiple_alignment_original.fa (4)` is the file including the original reference sequences

# 3. Standardize YFV FASTA definition line

In [6]:
p2yfrefseqs = pfs.data / 'ncbi/refsequences/yf/yf_2023_multiple_alignment_original.fa'
assert p2yfrefseqs.is_file()
fasta_original = FastaFileReader(p2yfrefseqs)
fasta_original.print_first_chunks(1)


Sequence 1:
>AY968064_Angola_1971
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAAT ...


/home/vtec/projects/bio/metagentorch/metagentorch/core.py:678: UserWarning: 
        None of the saved parsing rules were able to extract metadata from the first line in this file.
        You must set a custom rule (pattern + keys) before parsing text, by using:
            `self.set_parsing_rules(custom_pattern, custom_list_of_keys)`
                
  warnings.warn(msg, category=UserWarning)


Metadata on YFV in the NCBI database: 

- Taxonomy ID: [11089](https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=11089)
- Accession Number: [AY968064](https://www.ncbi.nlm.nih.gov/nuccore/AY968064)

Metadata on [AY968064](https://www.ncbi.nlm.nih.gov/nuccore/AY968064)
```
source: 
    /organism="Yellow fever virus"
    /mol_type="genomic RNA"
    /strain="Angola71"
    /host="Homo sapiens"/db_xref="taxon:11089"
    /geo_loc_name="Angola"

CDS:
    121..10359
    /codon_start=1
    /product="polyprotein"
    /protein_id="AAY34247.1"
```


In the original file, the definition line is like `>AY968064_Angola_1971` where the start of the line is the accession and the end is the strain name

We want to standardize it to match this project standard definition line (tab separated):
- `>taxonomy_id:ncbi:seqnb	seqnb	accession	taxonomy_id	ncbi accession  description`

YFV taxonomy ID is `11089` for all accessions. 

New definition line: `>11089:ncbi:seqnb accession	11089	ncbi	11098	strain`

`>AY968064_Angola_1971` will be transformed into `>11089:ncbi:seqnb seqnb  AY968064	11089	ncbi	Angola_1971`

In [7]:
p2yf_new_dfn_lines = pfs.data / 'ncbi/refsequences/yf/yf_2023_yellow_fever_aligned.fa'

In [14]:
re_accession = re.compile(r'^>(?P<accession>[A-Z]{1,2}\d*)(\_|\.\d)?_(?P<species>.*)$')
re_accession.groupindex.keys()

dict_keys(['accession', 'species'])

In [9]:
CREATE_NEW_FA = False
if CREATE_NEW_FA:
    print('>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971')   
    original_fa = FastaFileReader(p2yfrefseqs)
    with open(p2yf_new_dfn_lines, 'w') as fp:
        for i, refseq in enumerate(original_fa):
            dfn = refseq['definition line']
            accession = re_accession.search(dfn).group('accession')
            species = re_accession.search(dfn).group('species')
            new_dfn = f">11089:ncbi:{i+1}\t{i+1}\t{accession}\t11089\tncbi\t{species}"

            fp.write(f"{new_dfn}\n{refseq['sequence']}\n")
            # print(dfn)
            # print(new_dfn)
            # if i > 10: break
else:
    print('No new fasta file created')

No new fasta file created


When we create FastFileReader with the new file, the format is understood

In [15]:
new_fa = FastaFileReader(p2yf_new_dfn_lines)
new_fa.print_first_chunks(1)


Sequence 1:
>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAAT ...


In [16]:
new_fa.re_rule_name

'fasta_ncbi_std'

# 4. Remove gaps from reference sequences

The original fasta file (`yf_2023_yellow_fever_aligned.fa`) consists of **aligned** sequences from the coding region ([CDS](https://www.ncbi.nlm.nih.gov/books/NBK470040/)) of each accession. This means:
- All sequence strings in the file have the same length
- Sequences include gaps  (`-` characters) used to align the sequences with each other.

In [17]:
p2fa_aligned = pfs.data / 'ncbi/refsequences/yf/yf_2023_yellow_fever_aligned.fa'
assert p2fa_aligned.is_file()
p2fa_original = pfs.data / 'ncbi/refsequences/yf/yf_2023_yellow_fever.fa'

fa_aligned = FastaFileReader(p2fa_aligned)
for refseq in fa_aligned:
    if '-' in refseq['sequence']:
        print('gap character found in position',refseq['sequence'].index('-'), 'for:')
        print(refseq['definition line'])
        break

gap character found in position 7993 for:
>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971


For our purpose, we want to use unaligned sequences. Therefore we need to remove those `-` characters from the sequences, before we use them for read simulation.

In [18]:
CREATE_NEW_FA = False

if CREATE_NEW_FA:

    with open(p2fa_aligned) as fp_in:
        i, EOF = 0, False
        with open(p2fa_original, 'w') as fp_out:
            print(f"Saving new file in {p2fa_original.name}")
            while not EOF:
                line = fp_in.readline()
                if line == '': 
                    EOF = True
                elif line.startswith('>'):
                    fp_out.write(line)
                else:
                    fp_out.write(line.replace('-', ''))
                    # l2 = line.replace('-', '')
                    # print(len(line)-len(l2), 'gaps removed')
                # if i > 20: break

Read the cleaned up file

In [19]:
p2fa = pfs.data / 'ncbi/refsequences/yf/yf_2023_yellow_fever.fa'
assert p2fa.is_file()

fa = FastaFileReader(p2fa)

refseq = next(fa)
print(refseq['definition line'])
seq = refseq['sequence']
print(f"{len(seq):,d} bases in the sequence. Gap in sequence? {'-' in seq}")

>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971
10,234 bases in the sequence. Gap in sequence? False


Check that no `-` characters are present in the sequences

In [20]:
fa.reset_iterator()
for refseq in fa:
    if '-' in refseq['sequence']:
        print('gap character found in position',refseq['sequence'].index('-'), 'for:')
        print(refseq['definition line'])

# 5. Generate simulated reads

## Review the data in the input reference sequence FASTA file

In [21]:
p2yfv_refs = pfs.data / 'ncbi/refsequences/yf'
assert p2yfv_refs.is_dir()
files = files_in_tree(p2yfv_refs, pattern='yellow_fever')

refsequences
  |--yf
  |    |--yf_2023_yellow_fever.fa (0)
  |    |--yf_2023_yellow_fever_with_missing_bases.fa (1)
  |    |--yf_2023_yellow_fever_aligned.fa (2)


In [ ]:
p2inputs = pfs.data / 'ncbi/refsequences/yf/yf_2023_yellow_fever.fa'
assert p2fa.is_file()
fasta = FastaFileReader(p2inputs)
fasta.print_first_chunks(3)


Sequence 1:
>11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGAGGGGTTCGCTCCTTGTCAAACAAAAT ...

Sequence 2:
>11089:ncbi:2	2	U54798	11089	ncbi	Ivory_Coast_1982
ATGTCTGGTCGCAAAGCTCAGGGAAAAACCCTGGGCGTCAATATGGTTCGACGGGGAGTCCGCTCCTTGTCAAACAAAAT ...

Sequence 3:
>11089:ncbi:3	3	DQ235229	11089	ncbi	Ethiopia_1961
ATGTCTGGTCGAAAAGCTCAGGGTAAAACCCTGGGCGTCAATATGGTAAGACGGGGAGCACGCTCCTTGTCAAACAAAAT ...


In [ ]:
fasta.reset_iterator()
cum_length = 0
for i, refseq in enumerate(fasta):
    cum_length += len(refseq['sequence'])
nb_sequences = i+1
mean_length = int(cum_length / nb_sequences)
print(f"{nb_sequences:,d} sequences in the file, with an average length of {mean_length:,d} bases")

69 sequences in the file, with an average length of 10,227 bases


## ART Illumina: Single read simulation - 150 bp read

In [22]:
show_doc(ArtIllumina)

---

[source](https://github.com/vtecftwy/metagentorch/blob/main/metagentorch/art.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### ArtIllumina

>      ArtIllumina (path2app:str|pathlib.Path, input_dir:str|pathlib.Path,
>                   output_dir:str|pathlib.Path=None,
>                   app_in_system_path:bool=False)

*Class to handle all aspects of simulating sequencing with art_illumina*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path2app | str \| pathlib.Path |  | full path to art_illumina application on the system |
| input_dir | str \| pathlib.Path |  | full path to dir where input files are |
| output_dir | str \| pathlib.Path | None | full path to dir where to save output files, if different from input_dir |
| app_in_system_path | bool | False | whether `art_illumina` is in the system path or not |

In [23]:
p2inputs = p2fa.parent
assert p2inputs.is_dir()

p2simread_outputs = pfs.data / 'ncbi/simreads/yf'
assert p2simread_outputs.is_dir()

print(f"Reference sequences from <{p2inputs.absolute()}>")
print(f"Simreads will be saved in <{p2simread_outputs}>")

Reference sequences from </home/vtec/projects/bio/metagentorch/data/ncbi/refsequences/yf>
Simreads will be saved in </home/vtec/projects/bio/metagentorch/data/ncbi/simreads/yf>


In [24]:
art = ArtIllumina(
    path2app=Path('/usr/bin/art_illumina'), 
    input_dir=p2inputs, 
    output_dir=p2simread_outputs
    )

Ready to operate with art: /usr/bin/art_illumina
Input files from : /home/vtec/projects/bio/metagentorch/data/ncbi/refsequences/yf
Output files to :  /home/vtec/projects/bio/metagentorch/data/ncbi/simreads/yf


### Parameters for ART Illumina read simulation

In [ ]:
show_doc(art.sim_reads)

---

[source](https://github.com/vtecftwy/metagentools/blob/main/metagentools/art.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### ArtIllumina.sim_reads

>      ArtIllumina.sim_reads (input_file:str, output_seed:str,
>                             sim_type:str='single', read_length:int=150,
>                             fold:int=10, mean_read:int=None,
>                             std_read:int=None, ss:str='HS25',
>                             overwrite:bool=False, print_output:bool=True)

*Simulates reads with art_illumina. Output files saved in a separate directory*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| input_file | str |  | name of the fasta file to use as input |
| output_seed | str |  | seed to use for the output files |
| sim_type | str | single | type of read simmulation: 'single' or 'paired' |
| read_length | int | 150 | length of the read in bp |
| fold | int | 10 | fold |
| mean_read | int | None | mean length of the read for paired reads |
| std_read | int | None | std of the read length, for paired reads |
| ss | str | HS25 | quality profile to use for simulation, |
| overwrite | bool | False | overwrite existing output files if true, raise error if false |
| print_output | bool | True | if True, prints art ilumina's CLI output |

We will run a **single** read simulations with the following parameters:

`input_file`

Check which files are available:

In [25]:
art.list_all_input_files()

yf_1971_angola.fa
yf_2023_multiple_alignment_original.fa
yf_2023_yellow_fever.fa
yf_2023_yellow_fever_aligned.fa
yf_2023_yellow_fever_with_missing_bases.fa
yf_NC_002031_full_sequence.fa


Pick the fasta file with the cleaned up reference sequences

In [26]:
input_fname = 'yf_2023_yellow_fever.fa'

`fold`

Fold coverage, also known as sequencing depth or read depth, represents the average number of times each base in the reference genome is expected to be sequenced. For example:
- If you set -f 20, it means you're simulating a sequencing run that would cover each base in the reference genome an average of 20 times.
- If you set -f 100, it would simulate coverage where each base is sequenced an average of 100 times.

The fold coverage is an important parameter because it affects:
- The total number of reads generated: Higher fold coverage results in more reads.
- The likelihood of capturing rare variants or sequencing errors: Higher coverage generally improves the ability to detect rare variants and distinguish true variants from sequencing errors.
- The overall quality of the simulated dataset: Higher coverage typically leads to more accurate representation of the reference genome in the simulated data.

It's worth noting that ART Illumina uses this fold coverage value along with the read length and reference genome size to calculate the total number of reads to generate. The actual formula is:

```Total number of reads = (Genome size * Fold coverage) / Read length```

In [ ]:
fold = 250
genome_size = mean_length
nb_reads_per_sequence = (genome_size * fold) // 150
print(f"Estimated number of reads: {nb_reads_per_sequence:,d} per sequence")
print(f"Estimated total number of reads: {nb_reads_per_sequence * nb_sequences:,d}")

Estimated number of reads: 17,045 per sequence
Estimated total number of reads: 1,176,105


Set all simulation parameters:

In [ ]:
sim_params = {
    'input_file': input_fname,
    "sim_type": "single",
    "read_length": 150,
    'nb_sequences': nb_sequences,
    "fold": fold,
    'q_profile': 'HS25'
}
# add an output seed for the simread files based on the simulation parameters:
sim_params['output_seed'] = f"{sim_params['sim_type']}_{sim_params['nb_sequences']}seq_{sim_params['read_length']}bp"
sim_params

{'input_file': 'yf_2023_yellow_fever.fa',
 'sim_type': 'single',
 'read_length': 150,
 'nb_sequences': 69,
 'fold': 250,
 'q_profile': 'HS25',
 'output_seed': 'single_69seq_150bp'}

Run the simulation:

In [ ]:
art.sim_reads( 
    input_file=sim_params['input_file'],
    output_seed=sim_params['output_seed'],
    sim_type=sim_params['sim_type'],
    read_length=sim_params['read_length'],
    fold=sim_params['fold'],
    ss=sim_params['q_profile'],
    overwrite=True
)

return code:  0 


    ====================ART====================
             ART_Illumina (2008-2016)          
          Q Version 2.5.8 (June 6, 2016)       
     Contact: Weichun Huang <whduke@gmail.com> 
    -------------------------------------------

                  Single-end Simulation

Total CPU time used: 30.165

The random seed for the run: 1731415434

Parameters used during run
	Read Length:	150
	Genome masking 'N' cutoff frequency: 	1 in 150
	Fold Coverage:            250X
	Profile Type:             Combined
	ID Tag:                   

Quality Profile(s)
	First Read:   HiSeq 2500 Length 150 R1 (built-in profile) 

Output files

  FASTQ Sequence File:
	/home/vtec/projects/bio/metagentools/data/ncbi/simreads/yf/single_69seq_150bp/single_69seq_150bp.fq

  ALN Alignment File:
	/home/vtec/projects/bio/metagentools/data/ncbi/simreads/yf/single_69seq_150bp/single_69seq_150bp.aln




Check the generated output files:

In [ ]:
art.list_last_output_files()

single_69seq_150bp.fq
single_69seq_150bp.aln


We also can have access to all output files in the ArtIlluma output folder:

In [ ]:
art.list_all_output_files()

single_69seq_150bp
- single_69seq_150bp.fq
- single_69seq_150bp.aln


## Review simulation output files

In [ ]:
p2aln = pfs.data / 'ncbi/simreads/yf/single_69seq_150bp/single_69seq_150bp.aln'
assert p2aln.is_file()
aln = AlnFileReader(p2aln)

aln.re_rule_name

'aln_art_illumina_ncbi_std'

In [ ]:
aln.reset_iterator()
for i, read in enumerate(aln):
    pass
print(f"{i+1:,d} simulated reads")

1,160,343 simulated reads


In [ ]:
pprint(aln.parse_header_reference_sequences())

{'11089:ncbi:1': {'organism': 'Angola_1971',
                  'refseq_accession': 'AY968064',
                  'refseq_length': '10234',
                  'refseqid': '11089:ncbi:1',
                  'refseqnb': '1',
                  'refsource': 'ncbi',
                  'reftaxonomyid': '11089'},
 '11089:ncbi:10': {'organism': 'Peru_Hsapiens_2007',
                   'refseq_accession': 'GQ379163',
                   'refseq_length': '10231',
                   'refseqid': '11089:ncbi:10',
                   'refseqnb': '10',
                   'refsource': 'ncbi',
                   'reftaxonomyid': '11089'},
 '11089:ncbi:11': {'organism': 'Spain_Vaccine_2004',
                   'refseq_accession': 'DQ118157',
                   'refseq_length': '10231',
                   'refseqid': '11089:ncbi:11',
                   'refseqnb': '11',
                   'refsource': 'ncbi',
                   'reftaxonomyid': '11089'},
 '11089:ncbi:12': {'organism': 'Singapore_2017',
       

In [ ]:
art.list_last_output_files()

single_69seq_150bp.fq
single_69seq_150bp.aln


Overview of the reference sequences used to generate the simreads:

In [ ]:
p2aln = p2simread_outputs / 'single_69seq_150bp/single_69seq_150bp.aln'
assert p2aln.exists()
aln = AlnFileReader(p2aln)
print('\n'.join(aln.header['reference sequences']))

@SQ	11089:ncbi:1	1	AY968064	11089	ncbi	Angola_1971	10234
@SQ	11089:ncbi:2	2	U54798	11089	ncbi	Ivory_Coast_1982	10231
@SQ	11089:ncbi:3	3	DQ235229	11089	ncbi	Ethiopia_1961	10234
@SQ	11089:ncbi:4	4	AY572535	11089	ncbi	Gambia_2001	10231
@SQ	11089:ncbi:5	5	MF405338	11089	ncbi	Ghana_Hsapiens_1927	10231
@SQ	11089:ncbi:6	6	U21056	11089	ncbi	Senegal_1927	10231
@SQ	11089:ncbi:7	7	AY968065	11089	ncbi	Uganda_1948	10234
@SQ	11089:ncbi:8	8	JX898871	11089	ncbi	ArD114896_Senegal_1995	10231
@SQ	11089:ncbi:9	9	JX898872	11089	ncbi	Senegal_Aedes-aegypti_1995	10231
@SQ	11089:ncbi:10	10	GQ379163	11089	ncbi	Peru_Hsapiens_2007	10231
@SQ	11089:ncbi:11	11	DQ118157	11089	ncbi	Spain_Vaccine_2004	10231
@SQ	11089:ncbi:12	12	MF289572	11089	ncbi	Singapore_2017	10231
@SQ	11089:ncbi:13	13	KU978764	11089	ncbi	Sudan_Hsapiens_1941	10231
@SQ	11089:ncbi:14	14	JX898878	11089	ncbi	ArD181250_Senegal_2005	10231
@SQ	11089:ncbi:15	15	JX898879	11089	ncbi	ArD181676_Senegal_2005	10231
@SQ	11089:ncbi:16	16	JX898881	11089	ncbi	Senegal

# end of section